# Reinforcement Learning on 2048
Created 25.06.2025

In [1]:
import numpy as np
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipyevents import Event
import time
import gymnasium as gym
import torch

In [2]:
sys.path.insert(1, str(Path("..")))
import andreas2048
from andreas2048.game import *

In [3]:
from andreas2048 import gym2048
#env = gym.make("andreas_2048")
env = gym2048.Env2048()

In [4]:
raise RuntimeError("STOP")

RuntimeError: STOP

### 1 Interactive game

In [ ]:
%matplotlib widget

game = andreas2048.Game(shape=(4,4))
plot_arrows = False
stdout = sys.stdout

def on_press(e):
    global plot_arrows, stdout
    if not game.alive:
        return
    match e.key:
        case "left":
            r = game.try_move(Action.LEFT)
        case "right":
            r = game.try_move(Action.RIGHT)
        case "up":
            r = game.try_move(Action.UP)
        case "down":
            r = game.try_move(Action.DOWN)
        case "r":
            plot_arrows = not plot_arrows
        case "z":
            game.undo()
        case _:
            r = False
    game.plot_on_axis(ax, plot_arrows=plot_arrows)
    fig.canvas.draw()

fig, ax = plt.subplots()
fig.canvas.mpl_connect('key_press_event', on_press)
game.plot_on_axis(ax)
plt.show()
print("Test")

### 2 Random game

In [ ]:
n = 1000
score = []

t0 = time.perf_counter()
for i in range(n):
    game = Game()
    while game.alive:
        game.try_move(np.random.choice(np.array(game.get_moves())))
    score.append(game.score)
t1 = time.perf_counter()
print(f"Played {n} games in {(t1-t0):1.3f} s")

plt.hist(score)
plt.show()

In [ ]:
game = Game()

while game.alive:
    game.try_move(np.random.choice(np.array(game.get_moves())))

print(game)
ax = plt.subplot()
game.plot_on_axis(ax)
plt.show()

plt.plot(game.score_history)
plt.show()

### 3 Gymnasium

In [ ]:
from gymnasium.utils.env_checker import check_env

try:
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

##### 3.1 Testing some dummy policies

In [ ]:
class Random_Policy:

    def __init__(self, env: gym.Env) -> None:
        self.env = env

    def get_action(self, obs: np.ndarray) -> Action:
        return np.random.choice([Action.UP, Action.DOWN, Action.LEFT, Action.RIGHT])
    

class RDPolicy():
    def __init__(self, env: gym2048.Env2048) -> None:
        self.env = env

    def get_action(self, obs: np.ndarray) -> Action:
        moves = self.env.game.get_moves()
        for a in [Action.DOWN, Action.RIGHT, Action.LEFT, Action.UP]:
            if a in moves:
                return a

In [ ]:
class DQN_Agent:

    def __init__(self, env: gym.Env) -> None:
        self.env = env

    def get_action(self, obs: np.ndarray) -> Action:
        pass

    def build_model(self):
        self.model =  torch.nn.Sequential(
            torch.nn.Linear(4*4*16, out_features=128), 
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=128, out_features=128),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=128,out_features=4)
        )

In [ ]:
from matplotlib.ticker import FuncFormatter, MaxNLocator


agent = RDPolicy(env)
scores = []
highest_tiles = []
move_counts = []

for episode in range(100):
    # Start a new hand
    obs, info = env.reset()
    done = False

    # Play one complete hand
    while not done:
        # Agent chooses action (initially random, gradually more intelligent)
        action = agent.get_action(obs)

        # Take action and observe result
        next_obs, reward, terminated, truncated, info = env.step(action)

        # Learn from this experience
        #agent.update(obs, action, reward, terminated, next_obs)

        # Move to next state
        done = terminated or truncated
        obs = next_obs
    scores.append(env.game.score)
    highest_tiles.append(env.game.highest_tile)
    move_counts.append(env.game.move_count)

ax1 = plt.subplot()
ax2 = ax1.twinx()
ax3 = ax1.twinx()
ax3.spines.right.set_position(("axes", 1.2))
ax1.plot(scores, label="Scores")
p2 = ax2.plot(np.log2(highest_tiles), label="Highest tile", c="orange")
p3 = ax3.plot(move_counts, label="Move count", c="green")
ax1.set_xlabel("Episode")
ax1.set_ylabel("Score")
ax2.set_ylabel("Highest Tile")
ax3.set_ylabel("Move count")
ax2.yaxis.label.set_color(p2[0].get_color())
ax3.yaxis.label.set_color(p3[0].get_color())
ax2.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{2**x:n}"))
ax2.yaxis.set_major_locator(MaxNLocator(integer=True))
plt.show()